## Google Search versus Open Perplex

### Preparation

In [ ]:
PERPLEXITY_API_KEY = "your perplexity api key"
client_sync = OpenperplexSync(PERPLEXITY_API_KEY)

def extract_product_info(response_text):
    product_info = []
    # Split the response into blocks for each product (assuming each block starts with "Product Title:")
    product_blocks = response_text.strip().split('\n\n')
    for block in product_blocks:
        # Look for each product's information using the exact format
        title_match = re.search(r"Product Title:\s*(.*)", block)
        description_match = re.search(r"Description:\s*(.*)", block)
        brand_match = re.search(r"Brand:\s*(.*)", block)
        retailer_match = re.search(r"Retailer:\s*(.*)", block)
        source_match = re.search(r"Source URL:\s*(http[s]?://\S+)", block)
        comments_match = re.search(r"Comments:\s*(.*)", block)
        # Extract the details if they are found
        title = title_match.group(1) if title_match else "N/A"
        description = description_match.group(1) if description_match else "N/A"
        brand = brand_match.group(1) if brand_match else "N/A"
        retailer = retailer_match.group(1) if retailer_match else "N/A"
        source = source_match.group(1) if source_match else "N/A"
        comments = comments_match.group(1) if comments_match else "N/A"
        product_info.append({
            "Title": title,
            "Description": description,
            "Brand": brand,
            "Retailer": retailer,
            "Source": source,
            "Comments": comments
        })
    return product_info

def fetch_perplexity_products(query, csv_filename, product_num=10):
    """
    For a given query, fetch the top product_num best products using the Perplexity API,
    extract product information from the structured response, and save the results to a CSV file.

    Parameters:
        query (str): The base query string.
        csv_filename (str): The output CSV file name.
        product_num (int): The number of products to fetch. Default is 20.

    Returns:
        pd.DataFrame: DataFrame containing the extracted product info.
    """
    data = []
    # Build the full query prompt by inserting the query.
    full_query = f"""
    Provide the top {product_num} {query}.
    For each product, return the following details in this exact format:

    Product Title: [Title]
    Description: [Description]
    Brand: [Brand]
    Retailer: [Retailer]
    Source URL: [URL]
    Comments: [Comments]

    Make sure to return the details for all products in the exact format above, one after the other.
    IMPORTANT: Please include data for all these keys.
    """

    print(f"Perplexity Web Searching: {full_query}")

    try:
        response = client_sync.search(
            query=full_query,
            model='o3-mini-high',
            response_language="en",
            answer_type="text"
        )
        print("API Response:", response)
        # Get the LLM response text.
        response_text = response.get("llm_response", "")
        # Extract product information using your helper function.
        product_info = extract_product_info(response_text)
        # If product info was found, append it to data with a common timestamp.
        if product_info:
            timestamp = datetime.now(local_tz).strftime("%Y-%m-%d %H:%M:%S")
            for info in product_info:
                data.append({
                    "Search Source": "Perplexity",
                    "Title": info["Title"],
                    "Description": info["Description"],
                    "Brand": info["Brand"],
                    "Retailer": info["Retailer"],
                    "Source": info["Source"],
                    "Comments": info["Comments"],
                    "Query": query,
                    "Timestamp": timestamp
                })
            time.sleep(2)
    except Exception as e:
        print(f"Error during Perplexity API call for {query}: {e}")

    if data:
        df = pd.DataFrame(data)
        df.to_csv(csv_filename, index=False)
        print(f"CSV file saved successfully as '{csv_filename}'!")
    else:
        print("No data was collected. Please check your API key, network connection, and query parameters.")
        return pd.DataFrame()

In [ ]:
from urllib.parse import urlparse, urlunparse

def normalize_url(url):
    try:
        parsed = urlparse(url)
        clean_url = urlunparse((parsed.scheme, parsed.netloc, parsed.path.rstrip("/"), '', '', ''))
        return clean_url.lower()
    except:
        return str(url).lower().strip()

def compare_2_urls_by_category(googlefile, llmfile, plot=False):
    df1 = pd.read_csv(googlefile)
    df2 = pd.read_csv(llmfile)

    # Add a new column "Normalized URL" by applying the normalize_url function on "Source"
    df1["Normalized URL"] = df1["Source"].apply(normalize_url)
    df2["Normalized URL"] = df2["Source"].apply(normalize_url)

    all_categories = sorted(set(df1["Category"].unique()) | set(df2["Category"].unique()))
    summary = []

    for category in all_categories:
        # Filter rows by category.
        urls1 = df1[df1["Category"] == category]
        urls2 = df2[df2["Category"] == category]

        # Compute sets of normalized URLs.
        urls1_set = set(urls1["Normalized URL"].dropna().unique())
        urls2_set = set(urls2["Normalized URL"].dropna().unique())

        common = urls1_set & urls2_set
        only_in_1 = urls1_set - urls2_set
        only_in_2 = urls2_set - urls1_set

        total_unique = len(urls1_set | urls2_set)
        overlap_rate = len(common) / total_unique if total_unique > 0 else 0

        summary.append({
            "Category": category,
            "URLs in Google Search Result": len(urls1_set),
            "URLs in LLM result": len(urls2_set),
            "Common URLs": len(common),
            "Highest Source Rank of Common URLs in Google Search Results": urls1[urls1['Normalized URL'].isin(common)]['Source Rank'].min() if common else "N/A",
            "Overlap Rate": f"{round(overlap_rate, 2)*100}%"
        })

        print(f"\n📦 Category: {category}")
        print(f"✅ Common URLs: {len(common)}")
        if len(common) > 0:
            print(f"Highest Source Rank of the Common URLs in Google Search results: {urls1[urls1['Normalized URL'].isin(common)]['Source Rank'].min()}")

        print(f"\n🔴 Only in {googlefile} ({len(only_in_1)}):")
        print(
            urls1[urls1["Normalized URL"].isin(only_in_1)]
            [["Source", "Source Rank"]]
            .drop_duplicates(subset="Source")
            .sort_values("Source Rank")
        )

        print(f"\n🔵 Only in {llmfile} ({len(only_in_2)}):")
        print(
            urls2[urls2["Normalized URL"].isin(only_in_2)]
            # [["Source", "Source Rank"]]
            [["Source"]]
            .drop_duplicates(subset="Source")
            # .sort_values("Source Rank")
        )

        if plot:
            venn2([urls1_set, urls2_set], set_labels=("Google Search URLs", "Open Perplexity URLs"))
            plt.title(f"URL Overlap for {category}")
            plt.show()

    result_df = pd.DataFrame(summary)
    return result_df

### Five Categories

In [ ]:
local_tz = pytz.timezone("America/Toronto")

def save_serpapi_urls_to_csv(query, num_results=10, output_filename="serpapi_sources.csv"):
    timestamp = datetime.now(local_tz).strftime("%Y-%m-%d %H:%M:%S")
    urls = get_serpapi_urls(query, num_results=num_results)

    with open(output_filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Query", "Source", "Source Rank", "Timestamp", "Search Source"])

        for rank, url in enumerate(urls, start=1):
            writer.writerow([query, url, rank, timestamp, "Google Search"])

    print(f"✅ CSV file saved as: {output_filename}")

In [ ]:
query = "best instant coffee"
get_serpapi_urls(query, num_results=10)
# save_serpapi_urls_to_csv(query, num_results=10, output_filename="GoogleSearch_instant_coffee.csv")
# fetch_perplexity_products(query, csv_filename = "perplex_instant_coffee.csv", product_num=10)

['https://www.bonappetit.com/story/best-instant-coffee?srsltid=AfmBOoqJzh3ZyVIWuoVPxk9tQ8nPlw6roNwREo8E7X9j1nB-O66H5P2k',
 'https://www.reddit.com/r/Coffee/comments/y9cjhe/are_there_any_good_instant_coffees/',
 'https://www.tastingtable.com/1755382/instant-coffee-brands-ranked-worst-best/',
 'https://www.theguardian.com/food/2024/apr/10/best-instant-coffee-brands-supermarket-taste-test-australia',
 'https://sporked.com/article/best-instant-coffee/',
 'https://www.outdoorlife.com/gear/best-instant-coffee-for-backpacking/',
 'https://www.businessinsider.com/best-instant-coffee-at-grocery-store-taste-test-barista',
 'https://www.bogleheads.org/forum/viewtopic.php?t=433334',
 'https://www.costco.com/instant-coffee.html']

In [ ]:
query = "best baking ingredients"
save_serpapi_urls_to_csv(query, num_results=10, output_filename="GoogleSearch_baking_ingredients.csv")
# fetch_perplexity_products(query, csv_filename = "perplex_baking_ingredients.csv", product_num=10)

✅ CSV file saved as: GoogleSearch_baking_ingredients.csv


In [ ]:
query = "best cat food"
save_serpapi_urls_to_csv(query, num_results=10, output_filename="GoogleSearch_cat_food.csv")
# fetch_perplexity_products(query, csv_filename = "perplex_cat_food.csv", product_num=10)

✅ CSV file saved as: GoogleSearch_cat_food.csv


In [ ]:
query = "best dog food"
save_serpapi_urls_to_csv(query, num_results=10, output_filename="GoogleSearch_dog_food.csv")
# fetch_perplexity_products(query, csv_filename = "perplex_dog_food.csv", product_num=10)

✅ CSV file saved as: GoogleSearch_dog_food.csv
Perplexity Web Searching: 
    Provide the top 10 best dog food.
    For each product, return the following details in this exact format:

    Product Title: [Title]
    Description: [Description]
    Brand: [Brand]
    Retailer: [Retailer]
    Source URL: [URL]
    Comments: [Comments]

    Make sure to return the details for all products in the exact format above, one after the other.
    IMPORTANT: Please include data for all these keys.
    
API Response: {'llm_response': "Product Title: Open Farm Grain-Free Dry Dog Food  \nDescription: Premium dry dog food with humanely raised, grass-fed beef as the primary protein, offering a balanced nutritional profile with high protein and quality carbohydrates.  \nBrand: Open Farm  \nRetailer: Open Farm Online Store  \nSource URL: https://www.dogfoodadvisor.com/best-dog-foods/best-dry-dog-foods/  \nComments: Highly recommended for its ethical sourcing and excellent meat-to-vegetable balance, maki

In [ ]:
query = "best cat litter"
save_serpapi_urls_to_csv(query, num_results=10, output_filename="GoogleSearch_cat_litter.csv")
# fetch_perplexity_products(query, csv_filename = "perplex_cat_litter.csv", product_num=10)

✅ CSV file saved as: GoogleSearch_cat_litter.csv


### Merge five categories

In [ ]:
csv_files = glob.glob("GoogleSearch_*.csv")

df_list = []

for file in csv_files:
    df = pd.read_csv(file)

    # Remove prefix and suffix, then replace underscores with spaces.
    category_name = file.replace("GoogleSearch_", "").replace(".csv", "").replace("_", " ")
    df["Category"] = category_name
    df_list.append(df)

merged_perplexity_df = pd.concat(df_list, ignore_index=True)

merged_perplexity_df.to_csv("merged_GoogleSearch_Data_4_15_1800.csv", index=False)

print("All CSV files have been merged into 'merged_GoogleSearch_Data_4_15_1800.csv'.")

All CSV files have been merged into 'merged_GoogleSearch_Data_4_15_1800.csv'.


In [ ]:
# csv_files = glob.glob("perplex_*.csv")

# df_list = []

# for file in csv_files:
#     df = pd.read_csv(file)

#     # Remove prefix and suffix, then replace underscores with spaces.
#     category_name = file.replace("perplex_", "").replace(".csv", "").replace("_", " ")
#     df["Category"] = category_name
#     df_list.append(df)

# merged_perplexity_df = pd.concat(df_list, ignore_index=True)

# merged_perplexity_df.to_csv("merged_perplex_Data_4_8.csv", index=False)

# print("All CSV files have been merged into 'merged_perplex_Data_4_8.csv'.")

All CSV files have been merged into 'merged_perplex_Data_4_8.csv'.


### Comparison

In [ ]:
compare_2_urls_by_category('merged_GoogleSearch_Data_4_10.csv', 'merged_GoogleSearch_Data_4_15_1652.csv', plot=False)


📦 Category: baking ingredients
✅ Common URLs: 8
Highest Source Rank of the Common URLs in Google Search results: 1

🔴 Only in merged_GoogleSearch_Data_4_10.csv (0):
Empty DataFrame
Columns: [Source, Source Rank]
Index: []

🔵 Only in merged_GoogleSearch_Data_4_15_1652.csv (0):
Empty DataFrame
Columns: [Source]
Index: []

📦 Category: cat food
✅ Common URLs: 6
Highest Source Rank of the Common URLs in Google Search results: 2

🔴 Only in merged_GoogleSearch_Data_4_10.csv (3):
                                              Source  Source Rank
0  https://www.forbes.com/sites/forbes-personal-s...            1
5  https://www.reddit.com/r/CatAdvice/comments/1c...            6
7                    https://www.catfoodadvisor.com/            8

🔵 Only in merged_GoogleSearch_Data_4_15_1652.csv (2):
                                              Source
5  https://www.quora.com/Which-brands-of-cat-food...
6  https://www.youtube.com/playlist?list=PLQgNJka...

📦 Category: cat litter
✅ Common URLs: 8
Hig

,Category,URLs in Google Search Result,URLs in LLM result,Common URLs,Highest Source Rank of Common URLs in Google Search Results,Overlap Rate
0,baking ingredients,8,8,8,1,100.0%
1,cat food,9,8,6,2,55.00000000000001%
2,cat litter,9,8,8,1,89.0%
3,dog food,10,8,7,1,64.0%
4,instant coffee,9,9,8,1,80.0%
